In [1]:
pip install numpy yfinance requests feedparser beautifulsoup4 nltk vaderSentiment streamlit joblib matplotlib

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install joblib

In [45]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import joblib


In [77]:
# Load price data
price = pd.read_csv(r"E:\DEV\Data_science\DataScience\Assignments\stock-sentiment\data\AAPL_price.csv",encoding="latin1",parse_dates=["date"])

# Load RSS news
news = pd.read_csv(r"E:\DEV\Data_science\DataScience\Assignments\stock-sentiment\data\AAPL_news_rss.csv",  encoding="latin1",
    parse_dates=["date"])


In [78]:
print(price.head())
print(news.head())

   Unnamed: 0                publishedAt       date  \
0           0  2025-09-15T20:25:00+00:00 2025-09-15   
1           1  2025-09-15T16:27:00+00:00 2025-09-15   
2           2  2025-09-15T10:55:37+00:00 2025-09-15   
3           3  2025-09-13T12:43:00+00:00 2025-09-13   
4           4  2025-09-12T10:50:36+00:00 2025-09-12   

                      source  \
0  Investor's Business Daily   
1                   Barron's   
2              Yahoo Finance   
3            The Motley Fool   
4              Yahoo Finance   

                                               title  \
0  Apple Stock Rises Amid Strong iPhone 17 Preord...   
1  Apple Stock Gains. What Preorders Say About iP...   
2  Apple (AAPL) Valuation: Examining the Numbers ...   
3  Best Stock to Buy Now: Alphabet or Apple? - Th...   
4  Rosenblatt Lifts PT on Apple (AAPL) Stock - Ya...   

                                             summary  \
0  <a href="https://news.google.com/rss/articles/...   
1  <a href="https://news.go

In [79]:
price.columns

Index(['Unnamed: 0', 'publishedAt', 'date', 'source', 'title', 'summary',
       'url'],
      dtype='object')

In [80]:
news.columns

Index(['publishedAt', 'date', 'source', 'title', 'summary', 'url'], dtype='object')

In [81]:
analyzer = SentimentIntensityAnalyzer()

# Compute compound score from "title"
news["compound"] = news["title"].astype(str).apply(
    lambda x: analyzer.polarity_scores(x)["compound"]
)

# Aggregate daily sentiment
daily_sent = news.groupby("date").agg(
    sent_mean=("compound", "mean"),
    sent_count=("compound", "count")
).reset_index()


In [82]:
# Merge by "date"
price = price.merge(daily_sent, on="date", how="left")

# Fill missing sentiment
price[["sent_mean", "sent_count"]] = price[["sent_mean", "sent_count"]].fillna(0)

# Add lagged sentiment
price["sent_next"] = price["sent_mean"].shift(1)

# Drop rows with NA
price = price.dropna()
price.head()

,Unnamed: 0,publishedAt,date,source,title,summary,url,sent_mean,sent_count,sent_next
1,1,2025-09-15T16:27:00+00:00,2025-09-15,Barron's,Apple Stock Gains. What Preorders Say About iP...,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMihAFBV...,0.208188,40,0.208188
2,2,2025-09-15T10:55:37+00:00,2025-09-15,Yahoo Finance,Apple (AAPL) Valuation: Examining the Numbers ...,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMijAFBV...,0.208188,40,0.208188
3,3,2025-09-13T12:43:00+00:00,2025-09-13,The Motley Fool,Best Stock to Buy Now: Alphabet or Apple? - Th...,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMiigFBV...,0.161117,12,0.208188
4,4,2025-09-12T10:50:36+00:00,2025-09-12,Yahoo Finance,Rosenblatt Lifts PT on Apple (AAPL) Stock - Ya...,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMiggFBV...,0.108830,10,0.161117
5,5,2025-09-12T14:07:42+00:00,2025-09-12,Yahoo Finance,Apple Stock Downgraded After iPhone 17 Reveals...,"<a href=""https://news.google.com/rss/articles/...",https://news.google.com/rss/articles/CBMihgFBV...,0.108830,10,0.108830


In [84]:
features = ["sent_mean", "sent_count", "sent_next"]
X = price[features]

# Target: 1 if next day sentiment stronger, else 0
y = np.where(price["sent_mean"].shift(-1) > price["sent_mean"], 1, 0)

# Align sizes
X, y = X.iloc[:-1], y[:-1]

In [85]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))

Accuracy: 0.8
F1 Score: 0.6666666666666666


In [87]:
joblib.dump((model, features), "model.pkl")
print("✅ Model saved as models/model.pkl with features:", features)

✅ Model saved as models/model.pkl with features: ['sent_mean', 'sent_count', 'sent_next']
